In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re
from itertools import izip, islice

def parse_article(line):
    filtered_words = []
    
    try:
        # Parsing
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        
        for current_word, next_word in izip(words, islice(words, 1, None)):
            if current_word.lower() == 'narodnaya':
                found_str = "%s_%s" % (current_word.lower(), next_word.lower())
                filtered_words.append((found_str, 1))
        return filtered_words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part",16).map(parse_article)

In [2]:
result = wiki.filter(lambda x: len(x) > 0) \
    .flatMap(lambda xs: [(x[0], x[1]) for x in xs]) \
    .reduceByKey(lambda x,y:x+y) \
    .sortByKey(ascending=True)

In [3]:
result_list = [x for x in result.toLocalIterator()]

In [4]:
for result in result_list:
    print "%s\t%d" % (result[0], result[1])

narodnaya_gazeta	1
narodnaya_volya	9
